## The Data API

The main object used in the data API are ```datasets```. Usually you will use datasets that gradually read data from disk, but let's start with an example that creates a dataset entirely in RAM.

In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

```from_tensor_slices()``` takes a tensor and creates a Dataset whose elements are all slices of X (along the first dimension). This dataset contains 10 items: tensors 0, 1, 2, ..., 9.

In [4]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


## Chaining Transformations

Dataset methods reeturn a new Dataset, so we can chain transformations like so

In [5]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


Note that calling the repeat method, it returns a new dataset that repeats the items of the original dataset 3 times, however it does not copy data in memory 3 times!

batch() returned 2 remaining items in the last tensor, if we wanted to keep the tensors with the same size we could pass ```drop_remainder=True```

We can also apply transformations to each item with the ```map``` method. Sometimes we will perform expensive computations with this method, such as rotating or reshaping an image. To spawn multiple threads to speed things up, set the ```num_parallel_calls``` argument. Note that the function you pass to ```map``` must be convertible to a TF function.

In [7]:
for item in dataset.map(lambda x: x**2):
    print(item)

tf.Tensor([ 0  1  4  9 16 25 36], shape=(7,), dtype=int32)
tf.Tensor([49 64 81  0  1  4  9], shape=(7,), dtype=int32)
tf.Tensor([16 25 36 49 64 81  0], shape=(7,), dtype=int32)
tf.Tensor([ 1  4  9 16 25 36 49], shape=(7,), dtype=int32)
tf.Tensor([64 81], shape=(2,), dtype=int32)


The ```apply()```  methods applies a transformation to the dataset as whole. Using the ```unbatch``` function, each item in the new dataset will be a single-integer instead of a batch of seven items.

In [13]:
dataset = dataset.apply(tf.data.Dataset.unbatch)
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype

We can also filter data with ```filter```

In [14]:
dataset = dataset.filter(lambda x: x < 10)

And to look at a t few items from the dataset use ```take```

In [15]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


## Shuffling the data

The ```shuffle()``` method creates a new dataset that fills up a buffer with the first items of the source dataset. Whenever it is asked for an item, it will pull one out randomly from the buffer and replace it with a fresh new one from the source dataset, until it has iterated thorugh the source dataset. At this point it continues to  pull out items randomly from the buffer until it is empty.

You must specify the buffer size and it is important to make it large enough else shuffling will not be very effective

In [19]:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


For large datasets tha do not fit im memory, this simple shuffling-buffer approach may not be sufficient, since the buffer will be small compared to the size of the dataset.

One solution to this is to shuffle the source data itself, for example on Linux you can use ```shuf``` to shuffle text files. 
Even if the source data is shuffled, you migh want to shuffle it more or else the same order will be repeated at each epoch and the model might end up biased. To shuffle some more, a common approach is to split the source data into multiple files, then read them in a random order during training. With this, instances located in the same file will still end up close to each other. To avoid this you can pick multiple files randomly and read them simutaneously, interleaving their records.
Then on top of that we can add a shuffling buffer with ```shuffle```.

The best part about this: the Data API makes it easy for you to do all this.

### Interleaving lines from multiple files.